In [4]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for the YamboCalculator class

This tutorial describes the usage of the YamboCalculator class, for performing a single calculation
with the yambo.

## Perform an Hartree-Fock computation for Silicon

The first step needed to perform a Yambo computation is to generate the SAVE folder from a QuantumESPRESSO
computation.

The mppi.Utilities module has a function that perform this task

In [6]:
from mppi.Utilities import Utils as U

In [11]:
source_dir = 'QeCalculator_test/si_nscf_test1.save'
run_dir = 'YamboCalculator_test'

In [12]:
U.build_SAVE(source_dir,run_dir)

SAVE folder already present in YamboCalculator_test


Now the YamboInput class can create the yambo input object

In [13]:
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [1591.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [9377.0, 'RL'],
  'QPkrange': [[1, 13, 1, 8], '']}}

We restrict the analysis to the first two kpoints

In [14]:
inp['variables']['QPkrange'] = [[1,2,1,8],'']
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [1591.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [9377.0, 'RL'],
  'QPkrange': [[1, 2, 1, 8], '']}}

To run the computation we create an istance of YamboCalculator. This object behaves almost exactly as
the QeCalculator for what concerns the user interface

In [42]:
code = C.YamboCalculator(omp=1,mpi_run='mpirun -np 4',skip=True)

Initialize a Yambo calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 yambo


In [43]:
code.global_options()

{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'yambo',
 'skip': True,
 'verbose': True}

The structure of the folder in which yambo writes its results is governed by the name and jobname
variables. It is possible to provide only the name variable. 

The effect of this choice can be seen
in the command string executed by the calculator

In [46]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test1')
result

Run directory YamboCalculator_test
Skip the computation for input  hf_test1


{'output': ['YamboCalculator_test/hf_test1/o-hf_test1.hf'],
 'ndb': ['YamboCalculator_test/hf_test1/ndb.HF_and_locXC']}

In this case yambo create the hf_test1 folder and write all the file in this folder. Result is a dictionary with the name of the
o- file(s) and ndb database.

Instead, if we set also set a jobname we have

In [48]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test2',jobname = 'hf_job_test2')
result

Run directory YamboCalculator_test
Skip the computation for input  hf_test2


{'output': ['YamboCalculator_test/hf_test2/o-hf_job_test2.hf'],
 'ndb': ['YamboCalculator_test/hf_job_test2/ndb.HF_and_locXC']}

In this case yambo creates also the folder hf_job_test2 and write the database in this folder

Let see what happens if skip is set to False but the directory with the same name of the one that yambo
is going to create is found

In [49]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test1',skip=False)
result

Run directory YamboCalculator_test
delete folder: YamboCalculator_test/hf_test1
Executing command: mpirun -np 4 yambo -F hf_test1.in -J hf_test1 -C hf_test1


{'output': ['YamboCalculator_test/hf_test1/o-hf_test1.hf'],
 'ndb': ['YamboCalculator_test/hf_test1/ndb.HF_and_locXC']}

The YamboCalculator delete the hf_test1 folder before running the computation. 

In the same way, if also a jobname is specified, the skip = False option provides

In [51]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test2',jobname = 'hf_job_test2',skip=False)
result

Run directory YamboCalculator_test
delete folder: YamboCalculator_test/hf_test2
delete folder: YamboCalculator_test/hf_job_test2
Executing command: mpirun -np 4 yambo -F hf_test2.in -J hf_job_test2 -C hf_test2


{'output': ['YamboCalculator_test/hf_test2/o-hf_job_test2.hf'],
 'ndb': ['YamboCalculator_test/hf_job_test2/ndb.HF_and_locXC']}

## Perform a GW computation for Silicon

We make usage of the YamboCalculator to perform a different yambo computation. In this way we control
how this class manage the output files and the ndb database in various cases.

In [55]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp['variables']['QPkrange'] = [[1,2,1,8],'']
#inp

In [56]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test1')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F qp_test1.in -J qp_test1 -C qp_test1


{'output': ['YamboCalculator_test/qp_test1/o-qp_test1.qp'],
 'ndb': ['YamboCalculator_test/qp_test1/ndb.pp_fragment_13',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_2',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_4',
  'YamboCalculator_test/qp_test1/ndb.pp',
  'YamboCalculator_test/qp_test1/ndb.pp_fragment_11',
  'YamboCalculator_test/qp_test1/ndb.pp_fragment_9',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_9',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_12',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_8',
  'YamboCalculator_test/qp_test1/ndb.QP',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_13',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_6',
  'YamboCalculator_test/qp_test1/ndb.HF_and_locXC',
  'YamboCalculator_test/qp_test1/ndb.pp_fragment_2',
  'YamboCalculator_test/qp_test1/ndb.pp_fragment_4',
  'YamboCalculator_test/qp_test1/ndb.dip_iR_and_P_fragment_1',
  'YamboCalculator_test/

In [58]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test2',jobname = 'qp_job_test2')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F qp_test2.in -J qp_job_test2 -C qp_test2


{'output': ['YamboCalculator_test/qp_test2/o-qp_job_test2.qp'],
 'ndb': ['YamboCalculator_test/qp_job_test2/ndb.pp_fragment_13',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_2',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_4',
  'YamboCalculator_test/qp_job_test2/ndb.pp',
  'YamboCalculator_test/qp_job_test2/ndb.pp_fragment_11',
  'YamboCalculator_test/qp_job_test2/ndb.pp_fragment_9',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_9',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_12',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_8',
  'YamboCalculator_test/qp_job_test2/ndb.QP',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_13',
  'YamboCalculator_test/qp_job_test2/ndb.dip_iR_and_P_fragment_6',
  'YamboCalculator_test/qp_job_test2/ndb.HF_and_locXC',
  'YamboCalculator_test/qp_job_test2/ndb.pp_fragment_2',
  'YamboCalculator_test/qp_job_test2/ndb.pp_fragment_4',
  'YamboCalculator_test

We need to set the _get_ndb_names() method properly to extract only the database that we need.